# Evaluations

In [10]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
# notebook path get folder path, python path gets abs path
current_dir = Path.cwd().parent
os.chdir(current_dir)

import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

print(f"Current working directory is now: {Path.cwd()}")

from preprocess.data_preprocessing import *
from features.featuring import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current working directory is now: /Users/kevinb/Desktop/cse158/RSDB


In [11]:
url = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-California_10.json.gz"
meta_url = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/meta-California.json.gz"

cleaned_df = get_clean_review_data(url, meta_url)
featured_df = featuring_engineering(cleaned_df)
data_query = featured_df[['gmap_id', 'reviewer_id', 'rating']]

/Users/kevinb/Desktop/cse158/RSDB/rsdb
Loading metadata from: /Users/kevinb/Desktop/cse158/RSDB/rsdb/data/metadata.json.gz
Loaded 463034 metadata entries.
Processing review data from: /Users/kevinb/Desktop/cse158/RSDB/rsdb/data/data.json.gz


Processing chunks: 101it [01:17,  1.31it/s, Processed Rows=9581114]                         


Processed 9581114 review entries.
finished finding generalized categories. Takes 8.154163122177124
finished bining locations. Takes 9.085221290588379
finished featuring hours. Takes 49.04840016365051
finished creating model specalizied feature. Takes 22.112358808517456


In [17]:
train_df = featured_df.sample(frac=0.8, random_state=42)
test_df = featured_df.drop(train_df.index)

In [13]:
print(f"Current working directory is now: {Path.cwd()}")
model_path = 'trained_fpmc_model'
model = tf.keras.models.load_model(model_path)
print(model.summary())

Current working directory is now: /Users/kevinb/Desktop/cse158/RSDB
Model: "fpmc_variants"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  18724680  
                                                                 
 embedding_1 (Embedding)     multiple                  3968224   
                                                                 
 embedding_2 (Embedding)     multiple                  3968224   
                                                                 
 embedding_3 (Embedding)     multiple                  180045    
                                                                 
 embedding_4 (Embedding)     multiple                  38156     
                                                                 
 embedding_5 (Embedding)     multiple                  312       
                                                   

In [18]:
from train import fpmc_df_to_tf_dataset, fpmc_df_to_tf_dataset
# train_data = tlfm_df_to_tf_dataset(train_df).shuffle(1024).batch(4096)
# test_data = tlfm_df_to_tf_dataset(test_df).batch(4096)

train_data = fpmc_df_to_tf_dataset(train_df).shuffle(1024).batch(4096)
test_data = fpmc_df_to_tf_dataset(test_df).batch(4096)

In [19]:
test_metrics = model.evaluate(test_data, return_dict=True)
print(f"Test RMSE: {test_metrics['root_mean_squared_error']}")

365/365 [==============================] - 16s 44ms/step - loss: 0.0068 - root_mean_squared_error: 0.0000e+00
Test RMSE: 0.0


# Downstream Applications

In [25]:
from recommendation import *

ModuleNotFoundError: No module named 'tlfm'